In [ ]:
class Nut_profile:   
    '''
    class Nut_depth(resultpath,savepath,meshpath,ncfileDSi,ncfileDIN,ncfileDFe,first_year,last_year,
                 savefig=False,regional=True,runname)
                 
    c.f. Danilov et al. (2017):
    "in the vertical direction, the horizontal velocities and scalars are located at mid-levels" 
                 
    if regional = True, profiles will plotted for each main basins + Global Ocean. 
    Otherwise, just the Global Ocean.
    '''
    def __init__(self,resultpath,savepath,mesh,ncfileDSi,ncfileDIN,ncfileDFe,
                 first_year,last_year,savefig=False, regional=True, maxdepth = 6000, runname='fesom'):

        self.runname = runname
        self.resultpath = resultpath
        self.savepath = savepath
        self.mesh = mesh
        self.ncfileDSi = ncfileDSi
        self.ncfileDIN = ncfileDIN
        self.ncfileDFe = ncfileDFe
        self.fyear = first_year
        self.lyear = last_year
        self.savefig = savefig
        self.regional = regional
        self.maxdepth = maxdepth
        
        import matplotlib.pyplot as plt
        import numpy as np
        from netCDF4 import Dataset
        from scipy.interpolate import griddata
        import skill_metrics as sm
        import cartopy.crs as ccrs
        import pyfesom2 as pf
        from load_interp_WOA import WOAdata
        from load_interp_PISCES import PISCESdata
        
        years = np.arange(first_year, last_year+1,1)
        meshdiag = pf.get_meshdiag(mesh)
        runid      =  self.runname
        unitsDIN = 'DIN [mmol m$^{-3}$]'
        unitsDSi = 'DSi [mmol m$^{-3}$]'
        unitsDFe = 'DFe [mmol m$^{-3}$]'
        
        
        # load data -------------------------------------------------------------------------------------
        DINfesom = pf.get_data(resultpath, "DIN", years, mesh,
                               how="mean", compute=True, runid=runid, silent=True)

        DSifesom = pf.get_data(resultpath, "DSi", years, mesh,
                               how="mean", compute=True, runid=runid, silent=True)
        
        DFefesom = pf.get_data(resultpath, "DFe", years, mesh,
                               how="mean", compute=True, runid=runid, silent=True)

        DINwoa_input = WOAdata(runid,resultpath,mesh,ncfileDIN,'n_an', get_overview=False)
        DSiwoa_input = WOAdata(runid,resultpath,mesh,ncfileDSi,'i_an', get_overview=False)
        DFepisces_input = PISCESdata(runid,resultpath,mesh,ncfileDFe,'Fe', get_overview=False)
        
        DFepisces = DFepisces_input.pisces_int
        DFepisces[DFepisces>1000]= 0
        DFepisces[DFefesom == 0] = 0
        
        DINwoa = DINwoa_input.woa_int
        DINwoa[DINfesom == 0] = 0

        DSiwoa = DSiwoa_input.woa_int
        DSiwoa[DSifesom == 0] = 0
        
        # Load and derive profiles

        nod_area = np.ma.masked_equal(meshdiag.nod_area.values, 0)
        mask = pf.get_mask(mesh, "Global Ocean")

        DINfesom_by_area = ((np.ma.masked_equal(DINfesom[mask,:],0) * nod_area[:-1,:].T[mask]).mean(axis=0))
        DINwoa_by_area = ((np.ma.masked_equal(DINwoa[mask,:],0) * nod_area[:-1,:].T[mask]).mean(axis=0))

        DINfesom_weighted_Global = DINfesom_by_area/nod_area[:-1,:].T[mask].mean(axis=0)
        DINwoa_weighted_Global = DINwoa_by_area/nod_area[:-1,:].T[mask].mean(axis=0)

        DSifesom_by_area = ((np.ma.masked_equal(DSifesom[mask,:],0) * nod_area[:-1,:].T[mask]).mean(axis=0))
        DSiwoa_by_area = ((np.ma.masked_equal(DSiwoa[mask,:],0) * nod_area[:-1,:].T[mask]).mean(axis=0))

        DSifesom_weighted_Global = DSifesom_by_area/nod_area[:-1,:].T[mask].mean(axis=0)
        DSiwoa_weighted_Global = DSiwoa_by_area/nod_area[:-1,:].T[mask].mean(axis=0)
        
        DFefesom_by_area = ((np.ma.masked_equal(DFefesom[mask,:],0) * nod_area[:-1,:].T[mask]).mean(axis=0))
        DFepisces_by_area = ((np.ma.masked_equal(DFepisces[mask,:],0) * nod_area[:-1,:].T[mask]).mean(axis=0))

        DFefesom_weighted_Global = DFefesom_by_area/nod_area[:-1,:].T[mask].mean(axis=0)
        DFepisces_weighted_Global = DFepisces_by_area/nod_area[:-1,:].T[mask].mean(axis=0)
        
        if regional:
            mask = pf.get_mask(mesh, "Atlantic_Basin")

            DINfesom_by_area = ((np.ma.masked_equal(DINfesom[mask,:],0) * nod_area[:-1,:].T[mask]).mean(axis=0))
            DINwoa_by_area = ((np.ma.masked_equal(DINwoa[mask,:],0) * nod_area[:-1,:].T[mask]).mean(axis=0))

            DINfesom_weighted_Atlantic = DINfesom_by_area/nod_area[:-1,:].T[mask].mean(axis=0)
            DINwoa_weighted_Atlantic = DINwoa_by_area/nod_area[:-1,:].T[mask].mean(axis=0)

            DSifesom_by_area = ((np.ma.masked_equal(DSifesom[mask,:],0) * nod_area[:-1,:].T[mask]).mean(axis=0))
            DSiwoa_by_area = ((np.ma.masked_equal(DSiwoa[mask,:],0) * nod_area[:-1,:].T[mask]).mean(axis=0))

            DSifesom_weighted_Atlantic = DSifesom_by_area/nod_area[:-1,:].T[mask].mean(axis=0)
            DSiwoa_weighted_Atlantic = DSiwoa_by_area/nod_area[:-1,:].T[mask].mean(axis=0)
            
            DFefesom_by_area = ((np.ma.masked_equal(DFefesom[mask,:],0) * nod_area[:-1,:].T[mask]).mean(axis=0))
            DFepisces_by_area = ((np.ma.masked_equal(DFepisces[mask,:],0) * nod_area[:-1,:].T[mask]).mean(axis=0))

            DFefesom_weighted_Atlantic = DFefesom_by_area/nod_area[:-1,:].T[mask].mean(axis=0)
            DFepisces_weighted_Atlantic = DFepisces_by_area/nod_area[:-1,:].T[mask].mean(axis=0)

            mask = pf.get_mask(mesh, "Pacific_Basin")

            DINfesom_by_area = ((np.ma.masked_equal(DINfesom[mask,:],0) * nod_area[:-1,:].T[mask]).mean(axis=0))
            DINwoa_by_area = ((np.ma.masked_equal(DINwoa[mask,:],0) * nod_area[:-1,:].T[mask]).mean(axis=0))

            DINfesom_weighted_Pacific = DINfesom_by_area/nod_area[:-1,:].T[mask].mean(axis=0)
            DINwoa_weighted_Pacific = DINwoa_by_area/nod_area[:-1,:].T[mask].mean(axis=0)

            DSifesom_by_area = ((np.ma.masked_equal(DINfesom[mask,:],0) * nod_area[:-1,:].T[mask]).mean(axis=0))
            DSiwoa_by_area = ((np.ma.masked_equal(DINwoa[mask,:],0) * nod_area[:-1,:].T[mask]).mean(axis=0))

            DSifesom_weighted_Pacific = DSifesom_by_area/nod_area[:-1,:].T[mask].mean(axis=0)
            DSiwoa_weighted_Pacific = DSiwoa_by_area/nod_area[:-1,:].T[mask].mean(axis=0)
            
            DFefesom_by_area = ((np.ma.masked_equal(DFefesom[mask,:],0) * nod_area[:-1,:].T[mask]).mean(axis=0))
            DFepisces_by_area = ((np.ma.masked_equal(DFepisces[mask,:],0) * nod_area[:-1,:].T[mask]).mean(axis=0))

            DFefesom_weighted_Pacific = DFefesom_by_area/nod_area[:-1,:].T[mask].mean(axis=0)
            DFepisces_weighted_Pacific = DFepisces_by_area/nod_area[:-1,:].T[mask].mean(axis=0)

            mask = pf.get_mask(mesh, "Indian_Basin")

            DINfesom_by_area = ((np.ma.masked_equal(DINfesom[mask,:],0) * nod_area[:-1,:].T[mask]).mean(axis=0))
            DINwoa_by_area = ((np.ma.masked_equal(DINwoa[mask,:],0) * nod_area[:-1,:].T[mask]).mean(axis=0))

            DINfesom_weighted_Indian = DINfesom_by_area/nod_area[:-1,:].T[mask].mean(axis=0)
            DINwoa_weighted_Indian = DINwoa_by_area/nod_area[:-1,:].T[mask].mean(axis=0)

            DSifesom_by_area = ((np.ma.masked_equal(DSifesom[mask,:],0) * nod_area[:-1,:].T[mask]).mean(axis=0))
            DSiwoa_by_area = ((np.ma.masked_equal(DSiwoa[mask,:],0) * nod_area[:-1,:].T[mask]).mean(axis=0))

            DSifesom_weighted_Indian = DSifesom_by_area/nod_area[:-1,:].T[mask].mean(axis=0)
            DSiwoa_weighted_Indian = DSiwoa_by_area/nod_area[:-1,:].T[mask].mean(axis=0)
            
            DFefesom_by_area = ((np.ma.masked_equal(DFefesom[mask,:],0) * nod_area[:-1,:].T[mask]).mean(axis=0))
            DFepisces_by_area = ((np.ma.masked_equal(DFepisces[mask,:],0) * nod_area[:-1,:].T[mask]).mean(axis=0))

            DFefesom_weighted_Indian = DFefesom_by_area/nod_area[:-1,:].T[mask].mean(axis=0)
            DFepisces_weighted_Indian = DFepisces_by_area/nod_area[:-1,:].T[mask].mean(axis=0)
            
            mask = pf.get_mask(mesh, "Arctic_Basin")

            DINfesom_by_area = ((np.ma.masked_equal(DINfesom[mask,:],0) * nod_area[:-1,:].T[mask]).mean(axis=0))
            DINwoa_by_area = ((np.ma.masked_equal(DINwoa[mask,:],0) * nod_area[:-1,:].T[mask]).mean(axis=0))

            DINfesom_weighted_Arctic = DINfesom_by_area/nod_area[:-1,:].T[mask].mean(axis=0)
            DINwoa_weighted_Arctic = DINwoa_by_area/nod_area[:-1,:].T[mask].mean(axis=0)

            DSifesom_by_area = ((np.ma.masked_equal(DSifesom[mask,:],0) * nod_area[:-1,:].T[mask]).mean(axis=0))
            DSiwoa_by_area = ((np.ma.masked_equal(DSiwoa[mask,:],0) * nod_area[:-1,:].T[mask]).mean(axis=0))

            DSifesom_weighted_Arctic = DSifesom_by_area/nod_area[:-1,:].T[mask].mean(axis=0)
            DSiwoa_weighted_Arctic = DSiwoa_by_area/nod_area[:-1,:].T[mask].mean(axis=0)
            
            DFefesom_by_area = ((np.ma.masked_equal(DFefesom[mask,:],0) * nod_area[:-1,:].T[mask]).mean(axis=0))
            DFepisces_by_area = ((np.ma.masked_equal(DFepisces[mask,:],0) * nod_area[:-1,:].T[mask]).mean(axis=0))

            DFefesom_weighted_Arctic = DFefesom_by_area/nod_area[:-1,:].T[mask].mean(axis=0)
            DFepisces_weighted_Arctic = DFepisces_by_area/nod_area[:-1,:].T[mask].mean(axis=0)
            
            mask = pf.get_mask(mesh, "Southern_Ocean_Basin")

            DINfesom_by_area = ((np.ma.masked_equal(DINfesom[mask,:],0) * nod_area[:-1,:].T[mask]).mean(axis=0))
            DINwoa_by_area = ((np.ma.masked_equal(DINwoa[mask,:],0) * nod_area[:-1,:].T[mask]).mean(axis=0))

            DINfesom_weighted_Southern = DINfesom_by_area/nod_area[:-1,:].T[mask].mean(axis=0)
            DINwoa_weighted_Southern = DINwoa_by_area/nod_area[:-1,:].T[mask].mean(axis=0)

            DSifesom_by_area = ((np.ma.masked_equal(DSifesom[mask,:],0) * nod_area[:-1,:].T[mask]).mean(axis=0))
            DSiwoa_by_area = ((np.ma.masked_equal(DSiwoa[mask,:],0) * nod_area[:-1,:].T[mask]).mean(axis=0))

            DSifesom_weighted_Southern = DSifesom_by_area/nod_area[:-1,:].T[mask].mean(axis=0)
            DSiwoa_weighted_Southern = DSiwoa_by_area/nod_area[:-1,:].T[mask].mean(axis=0)
            
            DFefesom_by_area = ((np.ma.masked_equal(DFefesom[mask,:],0) * nod_area[:-1,:].T[mask]).mean(axis=0))
            DFepisces_by_area = ((np.ma.masked_equal(DFepisces[mask,:],0) * nod_area[:-1,:].T[mask]).mean(axis=0))

            DFefesom_weighted_Southern = DFefesom_by_area/nod_area[:-1,:].T[mask].mean(axis=0)
            DFepisces_weighted_Southern = DFepisces_by_area/nod_area[:-1,:].T[mask].mean(axis=0)
            
            # plotting
            
            fig, axs = plt.subplots(3,6, figsize=(14, 11), facecolor='w', edgecolor='k', constrained_layout=True, sharey=True)

            axs[0,0].plot(DINfesom_weighted_Global, mesh.zlev[:-1],label = 'FESOM', color = 'k', lw=3)
            axs[0,0].plot(DINwoa_weighted_Global, mesh.zlev[:-1],label = 'WOA', color = 'k', lw=3, linestyle = '--')
            axs[0,0].set_ylabel('Depth [km]',fontsize=14)
            axs[0,0].set_xlabel(unitsDIN,fontsize=14)
            axs[0,0].set_title('Global Ocean',size=16, weight='bold')
            axs[0,0].tick_params(labelsize=14)
            axs[0,0].grid()
            axs[0,0].legend(loc='best', borderaxespad=0., fontsize=14)
            
            axs[1,0].plot(DSifesom_weighted_Global, mesh.zlev[:-1],label = 'FESOM', color = 'k', lw=3)
            axs[1,0].plot(DSiwoa_weighted_Global, mesh.zlev[:-1],label = 'WOA', color = 'k', lw=3, linestyle = '--')
            axs[1,0].set_ylabel('Depth [km]',fontsize=14)
            axs[1,0].set_xlabel(unitsDSi,fontsize=14)
            axs[1,0].tick_params(labelsize=14)
            axs[1,0].grid()
            
            axs[2,0].plot(DFefesom_weighted_Global, mesh.zlev[:-1],label = 'FESOM', color = 'k', lw=3)
            axs[2,0].plot(DFepisces_weighted_Global, mesh.zlev[:-1],label = 'PISCES', color = 'k', lw=3, linestyle = '--')
            axs[2,0].set_ylabel('Depth [km]',fontsize=14)
            axs[2,0].set_xlabel(unitsDFe,fontsize=14)
            axs[2,0].tick_params(labelsize=14)
            axs[2,0].grid()
            axs[2,0].legend(loc='best', borderaxespad=0., fontsize=14)
            
            axs[0,1].plot(DINfesom_weighted_Pacific, mesh.zlev[:-1],label = 'FESOM', color = 'C0', lw=3)
            axs[0,1].plot(DINwoa_weighted_Pacific, mesh.zlev[:-1],label = 'WOA', color = 'C0', lw=3, linestyle = '--')
            axs[0,1].set_xlabel(unitsDIN,fontsize=14)
            axs[0,1].set_title('Pacific Ocean',size=16, weight='bold')
            axs[0,1].tick_params(labelsize=14)
            axs[0,1].grid()
            
            axs[1,1].plot(DSifesom_weighted_Pacific, mesh.zlev[:-1],label = 'FESOM', color = 'C0', lw=3)
            axs[1,1].plot(DSiwoa_weighted_Pacific, mesh.zlev[:-1],label = 'WOA', color = 'C0', lw=3, linestyle = '--')
            axs[1,1].set_xlabel(unitsDSi,fontsize=14)   
            axs[1,1].tick_params(labelsize=14)
            axs[1,1].grid()
            
            axs[2,1].plot(DFefesom_weighted_Pacific, mesh.zlev[:-1],label = 'FESOM', color = 'C0', lw=3)
            axs[2,1].plot(DFepisces_weighted_Pacific, mesh.zlev[:-1],label = 'WOA', color = 'C0', lw=3, linestyle = '--')
            axs[2,1].set_xlabel(unitsDFe,fontsize=14)   
            axs[2,1].tick_params(labelsize=14)
            axs[2,1].grid()
            
            axs[0,2].plot(DINfesom_weighted_Atlantic, mesh.zlev[:-1],label = 'FESOM', color = 'C1', lw=3)
            axs[0,2].plot(DINwoa_weighted_Atlantic, mesh.zlev[:-1],label = 'WOA', color = 'C1', lw=3, linestyle = '--')
            axs[0,2].set_xlabel(unitsDIN,fontsize=14)
            axs[0,2].set_title('Atlantic Ocean',size=16, weight='bold')
            axs[0,2].tick_params(labelsize=14)
            axs[0,2].grid()
            
            axs[1,2].plot(DSifesom_weighted_Atlantic, mesh.zlev[:-1],label = 'FESOM', color = 'C1', lw=3)
            axs[1,2].plot(DSiwoa_weighted_Atlantic, mesh.zlev[:-1],label = 'WOA', color = 'C1', lw=3, linestyle = '--')
            axs[1,2].set_xlabel(unitsDSi,fontsize=14)
            axs[1,2].tick_params(labelsize=14)
            axs[1,2].grid()
            
            axs[2,2].plot(DFefesom_weighted_Atlantic, mesh.zlev[:-1],label = 'FESOM', color = 'C1', lw=3)
            axs[2,2].plot(DFepisces_weighted_Atlantic, mesh.zlev[:-1],label = 'WOA', color = 'C1', lw=3, linestyle = '--')
            axs[2,2].set_xlabel(unitsDFe,fontsize=14)
            axs[2,2].tick_params(labelsize=14)
            axs[2,2].grid()
            
            axs[0,3].plot(DINfesom_weighted_Arctic, mesh.zlev[:-1],label = 'FESOM', color = 'C2', lw=3)
            axs[0,3].plot(DINwoa_weighted_Arctic, mesh.zlev[:-1],label = 'WOA', color = 'C2', lw=3, linestyle = '--')
            axs[0,3].set_xlabel(unitsDIN,fontsize=14)
            axs[0,3].set_title('Arctic Ocean',size=16, weight='bold')
            axs[0,3].tick_params(labelsize=14)
            axs[0,3].grid()
            
            axs[1,3].plot(DSifesom_weighted_Arctic, mesh.zlev[:-1],label = 'FESOM', color = 'C2', lw=3)
            axs[1,3].plot(DSiwoa_weighted_Arctic, mesh.zlev[:-1],label = 'WOA', color = 'C2', lw=3, linestyle = '--')
            axs[1,3].set_xlabel(unitsDSi,fontsize=14)
            axs[1,3].tick_params(labelsize=14)
            axs[1,3].grid()
            
            axs[2,3].plot(DFefesom_weighted_Arctic, mesh.zlev[:-1],label = 'FESOM', color = 'C2', lw=3)
            axs[2,3].plot(DFepisces_weighted_Arctic, mesh.zlev[:-1],label = 'WOA', color = 'C2', lw=3, linestyle = '--')
            axs[2,3].set_xlabel(unitsDFe,fontsize=14)
            axs[2,3].tick_params(labelsize=14)
            axs[2,3].grid()
            
            axs[0,4].plot(DINfesom_weighted_Southern, mesh.zlev[:-1],label = 'FESOM', color = 'C3', lw=3)
            axs[0,4].plot(DINwoa_weighted_Southern, mesh.zlev[:-1],label = 'WOA', color = 'C3', lw=3, linestyle = '--')
            axs[0,4].set_xlabel(unitsDIN,fontsize=14)
            axs[0,4].set_title('Southern Ocean',size=16, weight='bold')
            axs[0,4].tick_params(labelsize=14)
            axs[0,4].grid()
            
            axs[1,4].plot(DSifesom_weighted_Southern, mesh.zlev[:-1],label = 'FESOM', color = 'C3', lw=3)
            axs[1,4].plot(DSiwoa_weighted_Southern, mesh.zlev[:-1],label = 'WOA', color = 'C3', lw=3, linestyle = '--')
            axs[1,4].set_xlabel(unitsDSi,fontsize=14)
            axs[1,4].tick_params(labelsize=14)
            axs[1,4].grid()
            
            axs[2,4].plot(DFefesom_weighted_Southern, mesh.zlev[:-1],label = 'FESOM', color = 'C3', lw=3)
            axs[2,4].plot(DFepisces_weighted_Southern, mesh.zlev[:-1],label = 'WOA', color = 'C3', lw=3, linestyle = '--')
            axs[2,4].set_xlabel(unitsDFe,fontsize=14)
            axs[2,4].tick_params(labelsize=14)
            axs[2,4].grid()
            
            axs[0,5].plot(DINfesom_weighted_Indian, mesh.zlev[:-1],label = 'FESOM', color = 'C4', lw=3)
            axs[0,5].plot(DINwoa_weighted_Indian, mesh.zlev[:-1],label = 'WOA', color = 'C4', lw=3, linestyle = '--')
            axs[0,5].set_xlabel(unitsDIN,fontsize=14)
            axs[0,5].set_title('Indian Ocean',size=16, weight='bold')
            axs[0,5].tick_params(labelsize=14)
            axs[0,5].grid()
            
            axs[1,5].plot(DSifesom_weighted_Indian, mesh.zlev[:-1],label = 'FESOM', color = 'C4', lw=3)
            axs[1,5].plot(DSiwoa_weighted_Indian, mesh.zlev[:-1],label = 'WOA', color = 'C4', lw=3, linestyle = '--')
            axs[1,5].set_xlabel(unitsDSi,fontsize=14)
            axs[1,5].tick_params(labelsize=14)
            axs[1,5].grid()
            
            
            axs[2,5].plot(DFefesom_weighted_Indian, mesh.zlev[:-1],label = 'FESOM', color = 'C4', lw=3)
            axs[2,5].plot(DFepisces_weighted_Indian, mesh.zlev[:-1],label = 'WOA', color = 'C4', lw=3, linestyle = '--')
            axs[2,5].set_xlabel(unitsDFe,fontsize=14)
            axs[2,5].tick_params(labelsize=14)
            axs[2,5].grid()
            
            axs[0,0].set_ylim(-self.maxdepth,0)
            axs[1,0].set_ylim(-self.maxdepth,0)
            axs[2,0].set_ylim(-self.maxdepth,0)
            
        else:
            
            fig, axs = plt.subplots(1,3, figsize=(10, 5), facecolor='w', edgecolor='k', constrained_layout=True, sharey=True)

            axs[0].plot(DINfesom_weighted_Global, mesh.zlev[:-1],label = 'FESOM', color = 'k', lw=3)
            axs[0].plot(DINwoa_weighted_Global, mesh.zlev[:-1],label = 'WOA', color = 'k', lw=3, linestyle = '--')
            axs[0].set_ylabel('Depth [km]',fontsize=14)
            axs[0].set_xlabel(unitsDIN,fontsize=14)
            axs[1].set_title('Global Ocean',size=16, weight='bold')
            axs[0].tick_params(labelsize=14)
            axs[0].grid()
            axs[0].legend(loc='best', borderaxespad=0., fontsize=14)
            
            axs[1].plot(DSifesom_weighted_Global, mesh.zlev[:-1],label = 'FESOM', color = 'k', lw=3)
            axs[1].plot(DSiwoa_weighted_Global, mesh.zlev[:-1],label = 'WOA', color = 'k', lw=3, linestyle = '--')
            axs[1].set_ylabel('Depth [km]',fontsize=14)
            axs[1].set_xlabel(unitsDSi,fontsize=14)
            axs[1].tick_params(labelsize=14)
            axs[1].grid()
            
            axs[2].plot(DFefesom_weighted_Global, mesh.zlev[:-1],label = 'FESOM', color = 'k', lw=3)
            axs[2].plot(DFepisces_weighted_Global, mesh.zlev[:-1],label = 'PISCES', color = 'k', lw=3, linestyle = '--')
            axs[2].set_ylabel('Depth [km]',fontsize=14)
            axs[2].set_xlabel(unitsDFe,fontsize=14)
            axs[2].tick_params(labelsize=14)
            axs[2].grid()
            axs[2].legend(loc='best', borderaxespad=0., fontsize=14)
            
            axs[0].set_ylim(-self.maxdepth,0)
            axs[1].set_ylim(-self.maxdepth,0)
            axs[2].set_ylim(-self.maxdepth,0)
            
        if(savefig):
                plt.savefig(self.savepath+self.runname+'_'+'Nutrient_profiles_'+str(self.fyear)+'to'+str(self.lyear)+'.png', dpi = 300, bbox_inches='tight')
                plt.savefig(self.savepath+self.runname+'_'+'Nutrient_profiles_'+str(self.fyear)+'to'+str(self.lyear)+'.pdf', bbox_inches='tight')
        plt.show(block=False)